In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=0,1
import os, sys
import time
sys.path.insert(0, '..')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import lib
import torch, torch.nn as nn
import torch.nn.functional as F
from qhoptim.pyt import QHAdam

# read the data
data = pd.read_pickle('C:/Users/bcccu/Desktop/MQTTIDS/pkl/MQTTIDS_balanced.pkl')
# df = pd.read_csv('concatenated_data.csv')
print(f"Shape:",data['label'].value_counts())

device = 'cuda' if torch.cuda.is_available() else 'cpu'

env: CUDA_VISIBLE_DEVICES=0,1
Shape: label
Normal                             68890
encoded-payload-flood-Attack       40000
Connect-Disconnect-Flood-Attack    40000
publish-subscribe-flood-Attack     40000
Ping-Req-Flood-Attack              40000
slow-publish-Attack                40000
unvalid-publish-flood-Attack       40000
very-large-message-flood-Attack    40000
valid-publish-flood-Attack         40000
Name: count, dtype: int64


__Note:__ make sure you're using torch version `>= 1.1.0`, the code will silently fail even on 1.0.1.

In [2]:
df = data.copy()
# Remove duplicate rows
df.drop_duplicates(inplace=True)

print(" Shape after removing duplicates:", df['label'].value_counts())
categorical_features = df.select_dtypes(include=['object']).columns.tolist()
print("Categorical features:", categorical_features)

# Drop only the columns that exist in the dataframe
columns_to_drop = ['flowid', 'TCPid', 'srcIP', 'dstIP', 'srcPort', 'dstPort', 'clientID', 'clientIDlen', 'topic', 'startTime', 'endTime', 'username', 'password', 'fwdTobwdPacketRatio', 'bwdTofwdPacketRatio', 'willmsgLen', 'willtopicLen', 'protocolName', 'source_file']
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])
print("Columns dropped successfully! Shape:", df.shape)


 Shape after removing duplicates: label
Normal                             68882
encoded-payload-flood-Attack       40000
Connect-Disconnect-Flood-Attack    40000
publish-subscribe-flood-Attack     40000
Ping-Req-Flood-Attack              40000
slow-publish-Attack                40000
unvalid-publish-flood-Attack       40000
very-large-message-flood-Attack    40000
valid-publish-flood-Attack         40000
Name: count, dtype: int64
Categorical features: ['flowid', 'TCPid', 'srcIP', 'dstIP', 'clientID', 'topic', 'startTime', 'endTime', 'username', 'password', 'fwdTobwdPacketRatio', 'bwdTofwdPacketRatio', 'willmsgLen', 'willtopicLen', 'protocolName', 'label', 'source_file']
Columns dropped successfully! Shape: (388882, 387)


In [3]:
# Separate features and target
X = df.drop(columns=['label'])
y = df['label']

# Print dfset summary
print("df Cleaning Done! Shape after removing duplicates:", X.shape)
print("Label distribution:\n", y.value_counts())

# Fill missing testues
for col in X.columns:
    if X[col].isnull().sum() > 0:
        if X[col].dtype == 'object':  # Categorical
            X[col] = X[col].fillna(X[col].mode()[0])  # Assign explicitly
            print("Column:", col, "is categorical")
        else:  # Numerical
            X[col] = X[col].fillna(X[col].median())  # Assign explicitly

print("Missing testues handled successfully!")
print("Shape after handling missing values:", X.shape)

df Cleaning Done! Shape after removing duplicates: (388882, 386)
Label distribution:
 label
Normal                             68882
encoded-payload-flood-Attack       40000
Connect-Disconnect-Flood-Attack    40000
publish-subscribe-flood-Attack     40000
Ping-Req-Flood-Attack              40000
slow-publish-Attack                40000
unvalid-publish-flood-Attack       40000
very-large-message-flood-Attack    40000
valid-publish-flood-Attack         40000
Name: count, dtype: int64
Missing testues handled successfully!
Shape after handling missing values: (388882, 386)


In [4]:
data = df.copy()
# Separate features and target
X = data.drop(columns=['label'])
y = data['label']

# Print dataset summary
print("Data Cleaning Done! Shape after removing duplicates:", X.shape)
print("Label distribution:\n", y.value_counts())

# Fill missing testues
for col in X.columns:
    if X[col].isnull().sum() > 0:
        if X[col].dtype == 'object':  # Categorical
            X[col] = X[col].fillna(X[col].mode()[0])  # Assign explicitly
            print("Column:", col, "is categorical")
        else:  # Numerical
            X[col] = X[col].fillna(X[col].median())  # Assign explicitly

print("Missing testues handled successfully!")
print("Shape after handling missing values:", X.shape)

Data Cleaning Done! Shape after removing duplicates: (388882, 386)
Label distribution:
 label
Normal                             68882
encoded-payload-flood-Attack       40000
Connect-Disconnect-Flood-Attack    40000
publish-subscribe-flood-Attack     40000
Ping-Req-Flood-Attack              40000
slow-publish-Attack                40000
unvalid-publish-flood-Attack       40000
very-large-message-flood-Attack    40000
valid-publish-flood-Attack         40000
Name: count, dtype: int64
Missing testues handled successfully!
Shape after handling missing values: (388882, 386)


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Separate features and target
X = data.drop(columns=['label'])  # Assuming 'label' is the target column
y = data['label']

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data into 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42)

print("Data split completed!")
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

# Ensure the LabelEncoder is fitted before accessing classes_
if hasattr(label_encoder, 'classes_'):
	class_names = label_encoder.classes_
else:
	label_encoder.fit(y)  # Fit the LabelEncoder if not already fitted
	class_names = label_encoder.classes_

print("Class names:", class_names)

from sklearn.preprocessing import LabelEncoder

# Combine train and test sets for encoding
combined = pd.concat([X_train, X_test], axis=0)

# Encode categorical features
label_encoder = LabelEncoder()
for col in combined.select_dtypes(include=['object']).columns:
	combined[col] = label_encoder.fit_transform(combined[col].astype(str))

# Split back into train and test sets
X_train = combined.iloc[:X_train.shape[0], :]
X_test = combined.iloc[X_train.shape[0]:, :]


for col in X_train.columns:
    if X_train[col].isnull().sum() > 0:
        if X_train[col].dtype == 'object':  # Categorical
            X_train[col] = X_train[col].fillna(X_train[col].mode()[0])  # Assign explicitly
            X_test[col] = X_test[col].fillna(X_test[col].mode()[0])  # Assign explicitly
        else:  # Numerical
            X_train[col] = X_train[col].fillna(X_train[col].median())  # Assign explicitly
            X_test[col] = X_test[col].fillna(X_test[col].median())  # Assign explicitly



# Convert to Float32 for PyTorch compatibility
X_train, X_test = X_train.astype(np.float32), X_test.astype(np.float32)

Data split completed!
Train shape: (311105, 386), Test shape: (77777, 386)
Class names: ['Connect-Disconnect-Flood-Attack' 'Normal' 'Ping-Req-Flood-Attack'
 'encoded-payload-flood-Attack' 'publish-subscribe-flood-Attack'
 'slow-publish-Attack' 'unvalid-publish-flood-Attack'
 'valid-publish-flood-Attack' 'very-large-message-flood-Attack']


C:\Users\bcccu\AppData\Local\Temp\ipykernel_35596\1835829437.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col] = X_train[col].fillna(X_train[col].median())  # Assign explicitly
C:\Users\bcccu\AppData\Local\Temp\ipykernel_35596\1835829437.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[col] = X_test[col].fillna(X_test[col].median())  # Assign explicitly
C:\Users\bcccu\AppData\Local\Temp\ipykernel_35596\1835829437.py:51: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [ ]:
import numpy as np
from qhoptim.pyt import QHAdam  # This works
# Restore the deprecated alias temporarily
if not hasattr(np, 'int'):
    np.int = int
    np.float = float
    np.bool = bool

import lib  # Now import your library
import torch
import torch.nn as nn
import torch.nn.functional as F
import lib
from lib.trainer import Trainer
# Remove this line: from lib.qhadam import QHAdam
from qhoptim.pyt import QHAdam  # Use this instead

# Set up optimizer parameters
optimizer_params = {'nus': (0.7, 1.0), 'betas': (0.95, 0.998)}

ModuleNotFoundError: No module named 'lib.trainer'

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test_final = le.transform(y_test_final)
y_temp = le.transform(y_temp)
print("Classes:", y_train.shape)
print("Classes:", len(class_names))

NameError: name 'y_test_final' is not defined

In [25]:
num_classes = len(np.unique(y_train))  # should now be 10
print("Number of classes:", num_classes)


Number of classes: 10


In [26]:
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)


In [ ]:
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded = le.fit_transform(y)  # one single fit
class_names = le.classes_

from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

X_temp, X_test_final, y_temp, y_test_final = train_test_split(
    X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

X_train_final, X_valid, y_train_final, y_valid = train_test_split(
    X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=42
)


X_train_final = X_train_final.astype(np.float32)
X_valid = X_valid.astype(np.float32)
X_test_final = X_test_final.astype(np.float32)

for col in X_train.columns:
    if X_train[col].isnull().sum() > 0:
        if X_train[col].dtype == 'object':  # Categorical
            X_train[col] = X_train[col].fillna(X_train[col].mode()[0])  # Assign explicitly
            X_test[col] = X_test[col].fillna(X_test[col].mode()[0])  # Assign explicitly
            X_valid[col] = X_valid[col].fillna(X_valid[col].mode()[0])  # Assign explicitly
        else:  # Numerical
            X_train[col] = X_train[col].fillna(X_train[col].median())  # Assign explicitly
            X_test[col] = X_test[col].fillna(X_test[col].median())  # Assign explicitly
            X_valid[col] = X_valid[col].fillna(X_valid[col].median())  # Assign explicitly

# Convert to Float32 for PyTorch compatibility
X_train, X_test, X_valid = X_train.astype(np.float32), X_test.astype(np.float32), X_valid.astype(np.float32)
# Replace NaN with column median
X_train = X_train.fillna(X_train.median())
X_valid = X_valid.fillna(X_valid.median())
X_test = X_test.fillna(X_test.median())

# Replace Infs with max finite value in each column
X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(X_train.median())
X_valid = X_valid.replace([np.inf, -np.inf], np.nan).fillna(X_valid.median())
X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(X_test.median())




# ✅ 3. Define dataset wrapper
class CustomDataset:
    def __init__(self, X_train, y_train, X_valid, y_valid, X_test, y_test):
        self.X_train = X_train.values if hasattr(X_train, 'values') else X_train
        self.y_train = y_train if isinstance(y_train, np.ndarray) else y_train.values
        self.X_valid = X_valid.values if hasattr(X_valid, 'values') else X_valid
        self.y_valid = y_valid if isinstance(y_valid, np.ndarray) else y_valid.values
        self.X_test = X_test.values if hasattr(X_test, 'values') else X_test
        self.y_test = y_test if isinstance(y_test, np.ndarray) else y_test.values

# ✅ Replace NaNs and Infs in X_train_final, X_valid, X_test_final

for df in [X_train_final, X_valid, X_test_final]:
    df.replace([np.inf, -np.inf], np.nan, inplace=True)  # Replace inf with nan
    df.fillna(df.median(), inplace=True)  # Fill nan with median

X_train_final = X_train_final.astype(np.float32)
X_valid = X_valid.astype(np.float32)
X_test_final = X_test_final.astype(np.float32)
       

data = CustomDataset(X_train_final, y_train_final, X_valid, y_valid, X_test_final, y_test_final)

# ✅ 4. Define model
input_dim = data.X_train.shape[1]
num_classes = len(np.unique(data.y_train))  # should be 10


from rtdl import DenseBlock, Lambda, entmax15, entmoid15

model = nn.Sequential(
    DenseBlock(
        input_dim=input_dim,
        layer_dim=128,
        num_layers=3,
        tree_dim=num_classes,  # must match label count
        depth=8,
        flatten_output=False,
        choice_function=entmax15,
        bin_function=entmoid15
    ),
    Lambda(lambda x: x.mean(dim=-1))  # [B, C, D] → [B, C]
).to(device)

# ✅ 5. Initialize with dummy input (optional but recommended)
with torch.no_grad():
    _ = model(torch.as_tensor(data.X_train[:100], dtype=torch.float32, device=device))

# ✅ 6. Create Trainer
experiment_name = 'mqtt_classification_' + time.strftime("%Y.%m.%d_%H-%M-%S", time.gmtime())

trainer = Trainer(
    model=model,
    loss_function=F.cross_entropy,
    experiment_name=experiment_name,
    warm_start=False,
    Optimizer=QHAdam,
    optimizer_params=optimizer_params,
    verbose=True,
    n_last_checkpoints=5
)


AttributeError: module 'lib' has no attribute 'DenseBlock'

In [14]:
import shutil

src = "logs/mqtt_classification_2025.07.16_18-02-36/checkpoint_temp_11500.pth"
dst = "logs/mqtt_classification_2025.07.16_18-02-36/checkpoint_best.pth"

shutil.copyfile(src, dst)
print("✅ Checkpoint updated: checkpoint_best.pth now points to the best model.")


✅ Checkpoint updated: checkpoint_best.pth now points to the best model.


In [ ]:
loss_history = []
err_history = []
best_val_err = float('inf')
best_step = 0
early_stopping_rounds = 5000
report_frequency = 100

for batch in lib.iterate_minibatches(data.X_train, data.y_train, batch_size=1024, 
                                      shuffle=True, epochs=float('inf')):

    X_batch, y_batch = batch
    X_batch = torch.tensor(X_batch, dtype=torch.float32).to(device)
    y_batch = torch.tensor(y_batch, dtype=torch.long).to(device)

    metrics = trainer.train_on_batch(X_batch, y_batch, device=device)
    loss_history.append(metrics['loss'])

    if trainer.step % report_frequency == 0:
        trainer.save_checkpoint()
        trainer.average_checkpoints(out_tag='avg')
        trainer.load_checkpoint(tag='avg')

        err = trainer.evaluate_classification_error(
            data.X_valid, data.y_valid, device=device, batch_size=1024
        )

        if err < best_val_err:
            best_val_err = err
            best_step = trainer.stepca
            trainer.save_checkpoint(tag='best')

        err_history.append(err)
        trainer.load_checkpoint()  # Reload last checkpoint
        trainer.remove_old_temp_checkpoints()

        print("📉 Loss: %.5f" % metrics['loss'])
        print("🔍 Val Error Rate: %.5f" % err)

    if trainer.step > best_step + early_stopping_rounds:
        print(f'⏹️ Early Stopping after {early_stopping_rounds} steps without improvement.')
        print("🏆 Best step: ", best_step)
        print("✅ Best Val Error Rate: %.5f" % best_val_err)
        break


Saved logs/mqtt_classification_2025.07.16_18-02-36\checkpoint_temp_100.pth


c:\Users\bcccu\Desktop\Node-Git\node\notebooks\..\lib\trainer.py:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoints = [torch.load(path) for path in paths]
c:\User

Loaded logs/mqtt_classification_2025.07.16_18-02-36\checkpoint_avg.pth
Saved logs/mqtt_classification_2025.07.16_18-02-36\checkpoint_best.pth
Loaded logs/mqtt_classification_2025.07.16_18-02-36\checkpoint_temp_100.pth
📉 Loss: 5.75079
🔍 Val Error Rate: 0.92562
Saved logs/mqtt_classification_2025.07.16_18-02-36\checkpoint_temp_200.pth
Loaded logs/mqtt_classification_2025.07.16_18-02-36\checkpoint_avg.pth
Saved logs/mqtt_classification_2025.07.16_18-02-36\checkpoint_best.pth
Loaded logs/mqtt_classification_2025.07.16_18-02-36\checkpoint_temp_200.pth
📉 Loss: 5.55258
🔍 Val Error Rate: 0.90812
Saved logs/mqtt_classification_2025.07.16_18-02-36\checkpoint_temp_300.pth
Loaded logs/mqtt_classification_2025.07.16_18-02-36\checkpoint_avg.pth
Saved logs/mqtt_classification_2025.07.16_18-02-36\checkpoint_best.pth
Loaded logs/mqtt_classification_2025.07.16_18-02-36\checkpoint_temp_300.pth
📉 Loss: 5.35216
🔍 Val Error Rate: 0.88426
Saved logs/mqtt_classification_2025.07.16_18-02-36\checkpoint_temp_400

In [9]:
import shutil

src = "C:/Users/bcccu/Desktop/Node-Git/node/notebooks/logs/mqtt_classification_2025.07.16_18-02-36/checkpoint_temp_11500.pth"
dst = "C:/Users/bcccu/Desktop/Node-Git/node/notebooks/logs/mqtt_classification_2025.07.16_18-02-36/checkpoint_best.pth"

shutil.copyfile(src, dst)
print("✅ Checkpoint copied to checkpoint_best.pth")


✅ Checkpoint copied to checkpoint_best.pth


In [10]:
trainer.experiment_path = "C:/Users/bcccu/Desktop/Node-Git/node/notebooks/logs/mqtt_classification_2025.07.16_18-02-36"
trainer.load_checkpoint(tag='best')


NameError: name 'trainer' is not defined

In [ ]:
test_err = trainer.evaluate_classification_error(
    data.X_test, data.y_test, device=device, batch_size=1024
)
print(f"🧪 Final Test Error Rate: {test_err:.4f}")



🧪 Final Test Error Rate: 0.2399


In [42]:
import torch
import numpy as np

def predict(model, X, device, batch_size=1024):
	preds = []
	with torch.no_grad():
		for i in range(0, len(X), batch_size):
			xb = torch.tensor(X[i:i+batch_size], dtype=torch.float32).to(device)
			logits = model(xb)
			pred = logits.argmax(dim=1).cpu().numpy()
			preds.extend(pred)
	return np.array(preds)

preds = predict(trainer.model, data.X_test, device=device, batch_size=1024)
print("Predictions shape:", preds.shape)
print("Predictions:", preds[:10])  # Display first 10 predictions
print("Classification report:", classification_report(data.y_test, preds, target_names=class_names))




Predictions shape: (7086,)
Predictions: [5 5 0 1 0 5 5 3 7 4]
Classification report:                 precision    recall  f1-score   support

   bf_dos/ddos       0.42      0.62      0.50       600
    bruteforce       0.84      0.92      0.88       296
delay_dos/ddos       0.71      0.14      0.24       600
       malaria       0.94      0.96      0.95       600
     malformed       0.93      0.95      0.94       585
        normal       0.72      0.96      0.83      2400
       slowite       1.00      0.99      1.00       205
  sub_dos/ddos       0.92      0.82      0.86       600
  syn_dos/ddos       0.71      0.35      0.47       600
 will_dos/ddos       0.97      0.52      0.68       600

      accuracy                           0.76      7086
     macro avg       0.82      0.72      0.73      7086
  weighted avg       0.78      0.76      0.74      7086



In [ ]:
preds = trainer.predict(data.X_test, device=device, batch_size=1024)


In [103]:
import time
import numpy as np
from sklearn.model_selection import train_test_split

# 1. Create proper train/validation/test splits
X_temp, X_test_final, y_temp, y_test_final = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
)
X_train_final, X_valid, y_train_final, y_valid = train_test_split(
    X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=42
)

# 2. Create dataset object compatible with lib.Trainer
class CustomDataset:
    def __init__(self, X_train, y_train, X_valid, y_valid, X_test, y_test):
        self.X_train = X_train.values if hasattr(X_train, 'values') else X_train
        self.y_train = y_train if isinstance(y_train, np.ndarray) else y_train.values
        self.X_valid = X_valid.values if hasattr(X_valid, 'values') else X_valid
        self.y_valid = y_valid if isinstance(y_valid, np.ndarray) else y_valid.values
        self.X_test = X_test.values if hasattr(X_test, 'values') else X_test
        self.y_test = y_test if isinstance(y_test, np.ndarray) else y_test.values

data = CustomDataset(X_train_final, y_train_final, X_valid, y_valid, X_test_final, y_test_final)

# 3. Create model
input_dim = data.X_train.shape[1]
num_classes = len(np.unique(y_train))

model = nn.Sequential(
    lib.DenseBlock(
        input_dim=input_dim,
        layer_dim=64,              # Width of decision layers (can be tuned)
        num_layers=2,              # Depth of stacked blocks
        tree_dim=num_classes,      # Output classes = 10
        depth=6,                   # Tree depth
        flatten_output=False,      # Important: don't flatten
        choice_function=lib.entmax15,
        bin_function=lib.entmoid15
    ),
    lib.Lambda(lambda x: x.mean(dim=-1))  # [B, C, D] → [B, C] for F.cross_entropy
).to(device)



# Initialize model
with torch.no_grad():
    res = model(torch.as_tensor(data.X_train[:100], device=device))

# 4. Create trainer with Windows-safe experiment name
experiment_name = 'mqtt_classification'
timestamp = time.strftime("%Y.%m.%d_%H-%M-%S", time.gmtime())
experiment_name = f"{experiment_name}_{timestamp}"

trainer = Trainer(
    model=model, 
    loss_function=F.cross_entropy,  # For classification
    experiment_name=experiment_name,
    warm_start=False,
    Optimizer=QHAdam,
    optimizer_params=optimizer_params,
    verbose=True,
    n_last_checkpoints=5
)

c:\Users\bcccu\Desktop\Node-Git\node\notebooks\..\lib\odst.py:113: UserWarning: Data-aware initialization is performed on less than 1000 data points. This may cause instability.To avoid potential problems, run this model on a data batch with at least 1000 data samples.You can do so manually before training. Use with torch.no_grad() for memory efficiency.
  warn("Data-aware initialization is performed on less than 1000 data points. This may cause instability."


In [100]:
from tqdm import tqdm
from IPython.display import clear_output
loss_history, err_history = [], []
best_val_err = 1.0
best_step = 0
early_stopping_rounds = 10_000
report_frequency = 100

In [104]:
with torch.no_grad():
    out = model(torch.tensor(data.X_train, dtype=torch.float32, device=device))
    print("✅ Final Logits Shape:", out.shape)  # ✅ Should now be: [64, 10]


✅ Final Logits Shape: torch.Size([17004, 128])


In [98]:
for batch in lib.iterate_minibatches(data.X_train, data.y_train, batch_size=1024, 
                                                shuffle=True, epochs=float('inf')):
    X_batch, y_batch = batch
    # Ensure y_batch is class indices, not one-hot
    if hasattr(y_batch, 'shape') and y_batch.ndim > 1:
        y_batch = np.argmax(y_batch, axis=1)
    # If your model output has extra dimensions, squeeze it inside train_on_batch or here
    # If you control train_on_batch, ensure it returns loss computed as:
    # loss = F.cross_entropy(logits.squeeze(), y_batch)
    X_batch = torch.tensor(X_batch, dtype=torch.float32).to(device)
    y_batch = torch.tensor(y_batch, dtype=torch.long).to(device)

    metrics = trainer.train_on_batch(X_batch, y_batch, device=device)
    
    loss_history.append(metrics['loss'])

    if trainer.step % report_frequency == 0:
        trainer.save_checkpoint()
        trainer.average_checkpoints(out_tag='avg')
        trainer.load_checkpoint(tag='avg')
        err = trainer.evaluate_classification_error(
            data.X_valid, data.y_valid, device=device, batch_size=1024)
        
        if err < best_val_err:
            best_val_err = err
            best_step = trainer.step
            trainer.save_checkpoint(tag='best')
        
        err_history.append(err)
        trainer.load_checkpoint()  # last
        trainer.remove_old_temp_checkpoints()
            
        clear_output(True)
        # plt.figure(figsize=[12, 6])
        # plt.subplot(1, 2, 1)
        # # Convert tensors in loss_history to numpy for plotting
        # plt.plot([l.detach().cpu().numpy() if hasattr(l, 'detach') else l for l in loss_history])
        # plt.grid()
        # plt.subplot(1,2,2)
        # plt.plot([float(e) for e in err_history])
        # plt.grid()
        # plt.show()
        print("Loss %.5f" % (metrics['loss']))
        print("Val Error Rate: %0.5f" % (err))
        
    if trainer.step > best_step + early_stopping_rounds:
        print('BREAK. There is no improvment for {} steps'.format(early_stopping_rounds))
        print("Best step: ", best_step)
        print("Best Val Error Rate: %0.5f" % (best_val_err))
        break

Loss 1.38405
Val Error Rate: 0.47459


KeyboardInterrupt: 

In [50]:
from sklearn.metrics import classification_report, confusion_matrix
import torch
import numpy as np

# Set model to eval mode
trainer.load_checkpoint(tag='best')  # or use 'avg' if you averaged checkpoints
trainer.model.eval()

# Inference function
def predict(model, X, device, batch_size=1024):
    preds = []
    with torch.no_grad():
        for i in range(0, len(X), batch_size):
            xb = torch.tensor(X[i:i+batch_size], dtype=torch.float32).to(device)
            logits = model(xb)
            pred = logits.argmax(dim=1).cpu().numpy()
            preds.extend(pred)
    return np.array(preds)

# Run prediction on test set
y_pred = predict(trainer.model, data.X_test, device)

# If your labels were encoded using LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(data.y_train)  # Fit on training labels for correct mapping

# Classification Report
print("📊 Classification Report:")
print(classification_report(data.y_test, y_pred, target_names=le.classes_))

# Confusion Matrix
print("🧾 Confusion Matrix:")
print(confusion_matrix(data.y_test, y_pred))


FileNotFoundError: [Errno 2] No such file or directory: 'logs/mqtt_classification_2025.07.16_15-26-34\\checkpoint_best.pth'

In [ ]:
trainer.load_checkpoint(tag='best')
error_rate = trainer.evaluate_classification_error(data.X_test, data.y_test, device=device, batch_size=1024)
print('Best step: ', trainer.step)
print("Test Error rate: %0.5f" % (error_rate))
trainer.load_checkpoint()

NameError: name 'trainer' is not defined